# ML Dynamic Pricing - Phase 1: Rule-Based Markdown Engine

Implements rule-based pricing recommendations to prepare for future ML enhancements.

## Business Rules
- **Age-based markdowns**: Products markdown based on days since receipt
- **Inventory triggers**: Overstocked items get price reductions
- **Seasonal clearance**: End-of-season products marked down

## Constraints
- Min Margin: 15% (price must be >= cost * 1.15)
- Max Price: MSRP (cannot exceed manufacturer suggested price)
- Max Change/Week: 10% (limit price volatility)
- Min Duration: 7 days (avoid frequent changes)

## Data Flow
```
Silver (dim_products, inventory_position_current) --> Gold (gold_pricing_recommendations)
```

## Usage
Schedule this notebook to run **daily** via Fabric pipeline.

## Output Schema
The `gold_pricing_recommendations` table includes:
- Current price and recommended price
- Reason codes for transparency
- Constraint validation flags
- Extensible schema for future ML scores

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.utils import AnalysisException
from datetime import datetime, timezone, timedelta
import os

In [ ]:
# =============================================================================
# PARAMETERS
# =============================================================================

def get_env(var_name, default=None):
    return os.environ.get(var_name, default)

SILVER_DB = get_env("SILVER_DB", default="ag")
GOLD_DB = get_env("GOLD_DB", default="au")

# Pricing constraints (business rules)
MIN_MARGIN_PCT = 0.15  # 15% minimum margin
MAX_PRICE_FACTOR = 1.0  # Max price = MSRP
MAX_CHANGE_PCT_WEEKLY = 0.10  # Max 10% change per week
MIN_DURATION_DAYS = 7  # Min 7 days between price changes

# Markdown rules
AGE_THRESHOLD_MODERATE = 30  # Days - start moderate markdown
AGE_THRESHOLD_AGGRESSIVE = 60  # Days - aggressive markdown
AGE_MARKDOWN_MODERATE = 0.10  # 10% markdown
AGE_MARKDOWN_AGGRESSIVE = 0.25  # 25% markdown

INVENTORY_HIGH_THRESHOLD = 100  # Units - considered overstocked
INVENTORY_MARKDOWN = 0.15  # 15% markdown for overstocked

# Seasonal clearance (example: end of Q4 for winter items)
CLEARANCE_MARKDOWN = 0.30  # 30% markdown for clearance

print(f"Configuration: SILVER_DB={SILVER_DB}, GOLD_DB={GOLD_DB}")
print(f"Constraints: Min Margin={MIN_MARGIN_PCT*100}%, Max Price=MSRP, Max Change/Week={MAX_CHANGE_PCT_WEEKLY*100}%")

In [ ]:
# =============================================================================
# HELPER FUNCTIONS
# =============================================================================

def ensure_database(name):
    spark.sql(f"CREATE DATABASE IF NOT EXISTS {name}")

def read_silver(table_name):
    return spark.table(f"{SILVER_DB}.{table_name}")

def read_gold(table_name):
    try:
        return spark.table(f"{GOLD_DB}.{table_name}")
    except AnalysisException:
        return None

def save_gold(df, table_name):
    full_name = f"{GOLD_DB}.{table_name}"
    df.write.format("delta").mode("overwrite").saveAsTable(full_name)
    print(f"  {full_name}: {df.count()} rows")

def silver_exists(table_name):
    try:
        spark.table(f"{SILVER_DB}.{table_name}")
        return True
    except AnalysisException:
        return False

ensure_database(GOLD_DB)

In [ ]:
# =============================================================================
# PRICING CONSTRAINTS TABLE
# =============================================================================

print("="*60)
print("CREATING PRICING CONSTRAINTS TABLE")
print("="*60)

# Create constraints reference table for transparency and configurability
constraints_data = [
    {"constraint_name": "min_margin_pct", "constraint_value": MIN_MARGIN_PCT, "description": "Minimum margin percentage (price >= cost * 1.15)"},
    {"constraint_name": "max_price_factor", "constraint_value": MAX_PRICE_FACTOR, "description": "Maximum price as factor of MSRP (1.0 = cannot exceed MSRP)"},
    {"constraint_name": "max_change_pct_weekly", "constraint_value": MAX_CHANGE_PCT_WEEKLY, "description": "Maximum price change per week (as percentage)"},
    {"constraint_name": "min_duration_days", "constraint_value": float(MIN_DURATION_DAYS), "description": "Minimum days between price changes"},
    {"constraint_name": "age_threshold_moderate", "constraint_value": float(AGE_THRESHOLD_MODERATE), "description": "Days to trigger moderate markdown"},
    {"constraint_name": "age_threshold_aggressive", "constraint_value": float(AGE_THRESHOLD_AGGRESSIVE), "description": "Days to trigger aggressive markdown"},
    {"constraint_name": "age_markdown_moderate", "constraint_value": AGE_MARKDOWN_MODERATE, "description": "Markdown percentage for moderate age threshold"},
    {"constraint_name": "age_markdown_aggressive", "constraint_value": AGE_MARKDOWN_AGGRESSIVE, "description": "Markdown percentage for aggressive age threshold"},
    {"constraint_name": "inventory_high_threshold", "constraint_value": float(INVENTORY_HIGH_THRESHOLD), "description": "Units threshold for overstocked items"},
    {"constraint_name": "inventory_markdown", "constraint_value": INVENTORY_MARKDOWN, "description": "Markdown percentage for overstocked items"},
    {"constraint_name": "clearance_markdown", "constraint_value": CLEARANCE_MARKDOWN, "description": "Markdown percentage for seasonal clearance"},
]

df_constraints = spark.createDataFrame(constraints_data)
save_gold(df_constraints, "pricing_constraints")
print()

In [ ]:
# =============================================================================
# LOAD SOURCE DATA
# =============================================================================

print("="*60)
print("LOADING SOURCE DATA")
print("="*60)

if not silver_exists("dim_products"):
    raise Exception("dim_products table not found. Run 02-historical-data-load.ipynb first.")

# Load product master with pricing data
df_products = read_silver("dim_products").select(
    F.col("ID").alias("product_id"),
    "ProductName",
    "Department",
    "Category",
    "Subcategory",
    "Cost",
    "MSRP",
    "SalePrice",
    "Tags"
)

print(f"Products loaded: {df_products.count()}")

# Load current inventory positions (aggregated across all stores)
df_inventory = None
if silver_exists("fact_store_inventory_txn"):
    # Calculate current inventory by product across all stores
    window_spec = Window.partitionBy("product_id").orderBy(F.desc("event_ts"))
    df_inventory = (
        read_silver("fact_store_inventory_txn")
        .withColumn("rn", F.row_number().over(window_spec))
        .filter(F.col("rn") == 1)
        .groupBy("product_id")
        .agg(
            F.sum("balance").alias("total_inventory"),
            F.max("event_ts").alias("inventory_as_of")
        )
    )
    print(f"Inventory positions loaded: {df_inventory.count()}")
else:
    print("No inventory data available - using product data only")

# Load previous pricing recommendations to check duration constraint
df_previous_prices = read_gold("gold_pricing_recommendations")
if df_previous_prices:
    print(f"Previous pricing recommendations loaded: {df_previous_prices.count()}")
else:
    print("No previous pricing recommendations - first run")

print()

In [ ]:
# =============================================================================
# CALCULATE PRODUCT AGE
# =============================================================================

print("="*60)
print("CALCULATING PRODUCT AGE")
print("="*60)

# For this phase, simulate product age based on receipt data
# In a real system, this would come from a product lifecycle table
df_product_age = None

if silver_exists("fact_receipt_lines"):
    # Calculate days since first receipt for each product
    df_product_age = (
        read_silver("fact_receipt_lines")
        .groupBy("product_id")
        .agg(
            F.min("event_ts").alias("first_receipt_ts"),
            F.max("event_ts").alias("last_receipt_ts")
        )
        .withColumn("current_ts", F.current_timestamp())
        .withColumn(
            "days_since_first_receipt",
            F.datediff(F.col("current_ts"), F.col("first_receipt_ts"))
        )
        .withColumn(
            "days_since_last_receipt",
            F.datediff(F.col("current_ts"), F.col("last_receipt_ts"))
        )
        .select("product_id", "days_since_first_receipt", "days_since_last_receipt")
    )
    print(f"Product age calculated for {df_product_age.count()} products")
else:
    print("No receipt data - product age will be 0")

print()

In [ ]:
# =============================================================================
# APPLY RULE-BASED MARKDOWN LOGIC
# =============================================================================

print("="*60)
print("APPLYING RULE-BASED MARKDOWN LOGIC")
print("="*60)

# Start with product master
df_pricing = df_products

# Join with inventory if available
if df_inventory is not None:
    df_pricing = df_pricing.join(df_inventory, "product_id", "left")
else:
    df_pricing = df_pricing.withColumn("total_inventory", F.lit(None).cast("long"))
    df_pricing = df_pricing.withColumn("inventory_as_of", F.lit(None).cast("timestamp"))

# Join with product age if available
if df_product_age is not None:
    df_pricing = df_pricing.join(df_product_age, "product_id", "left")
else:
    df_pricing = df_pricing.withColumn("days_since_first_receipt", F.lit(0))
    df_pricing = df_pricing.withColumn("days_since_last_receipt", F.lit(0))

# Initialize markdown factors and reason codes
df_pricing = df_pricing.withColumn("markdown_factor", F.lit(0.0))
df_pricing = df_pricing.withColumn("reason_codes", F.array())

# Rule 1: Age-based markdowns
df_pricing = df_pricing.withColumn(
    "age_markdown",
    F.when(
        F.col("days_since_last_receipt") >= AGE_THRESHOLD_AGGRESSIVE,
        F.lit(AGE_MARKDOWN_AGGRESSIVE)
    ).when(
        F.col("days_since_last_receipt") >= AGE_THRESHOLD_MODERATE,
        F.lit(AGE_MARKDOWN_MODERATE)
    ).otherwise(F.lit(0.0))
)

df_pricing = df_pricing.withColumn(
    "markdown_factor",
    F.greatest(F.col("markdown_factor"), F.col("age_markdown"))
)

df_pricing = df_pricing.withColumn(
    "reason_codes",
    F.when(
        F.col("age_markdown") == AGE_MARKDOWN_AGGRESSIVE,
        F.array_union(F.col("reason_codes"), F.array(F.lit("AGE_AGGRESSIVE")))
    ).when(
        F.col("age_markdown") == AGE_MARKDOWN_MODERATE,
        F.array_union(F.col("reason_codes"), F.array(F.lit("AGE_MODERATE")))
    ).otherwise(F.col("reason_codes"))
)

# Rule 2: Inventory-based markdowns
df_pricing = df_pricing.withColumn(
    "inventory_markdown",
    F.when(
        F.col("total_inventory") >= INVENTORY_HIGH_THRESHOLD,
        F.lit(INVENTORY_MARKDOWN)
    ).otherwise(F.lit(0.0))
)

df_pricing = df_pricing.withColumn(
    "markdown_factor",
    F.greatest(F.col("markdown_factor"), F.col("inventory_markdown"))
)

df_pricing = df_pricing.withColumn(
    "reason_codes",
    F.when(
        F.col("inventory_markdown") > 0,
        F.array_union(F.col("reason_codes"), F.array(F.lit("INVENTORY_HIGH")))
    ).otherwise(F.col("reason_codes"))
)

# Rule 3: Seasonal clearance (example: products tagged with winter/holiday in Q1)
current_month = datetime.now(timezone.utc).month
is_clearance_period = current_month in [1, 2, 3]  # Q1 for winter clearance

if is_clearance_period:
    df_pricing = df_pricing.withColumn(
        "seasonal_markdown",
        F.when(
            (F.col("Tags").isNotNull()) & 
            ((F.lower(F.col("Tags")).contains("winter")) | 
             (F.lower(F.col("Tags")).contains("holiday"))),
            F.lit(CLEARANCE_MARKDOWN)
        ).otherwise(F.lit(0.0))
    )
    
    df_pricing = df_pricing.withColumn(
        "markdown_factor",
        F.greatest(F.col("markdown_factor"), F.col("seasonal_markdown"))
    )
    
    df_pricing = df_pricing.withColumn(
        "reason_codes",
        F.when(
            F.col("seasonal_markdown") > 0,
            F.array_union(F.col("reason_codes"), F.array(F.lit("SEASONAL_CLEARANCE")))
        ).otherwise(F.col("reason_codes"))
    )

# Calculate recommended price before constraints
df_pricing = df_pricing.withColumn(
    "recommended_price_raw",
    F.col("SalePrice") * (1 - F.col("markdown_factor"))
)

print(f"Markdown logic applied to {df_pricing.count()} products")
print()

In [ ]:
# =============================================================================
# APPLY BUSINESS CONSTRAINTS
# =============================================================================

print("="*60)
print("APPLYING BUSINESS CONSTRAINTS")
print("="*60)

# Constraint 1: Minimum margin (price >= cost * 1.15)
df_pricing = df_pricing.withColumn(
    "min_price",
    F.col("Cost") * (1 + MIN_MARGIN_PCT)
)

# Constraint 2: Maximum price (cannot exceed MSRP)
df_pricing = df_pricing.withColumn(
    "max_price",
    F.col("MSRP") * MAX_PRICE_FACTOR
)

# Apply min/max constraints
df_pricing = df_pricing.withColumn(
    "recommended_price",
    F.greatest(
        F.col("min_price"),
        F.least(F.col("recommended_price_raw"), F.col("max_price"))
    )
)

# Track constraint violations
df_pricing = df_pricing.withColumn(
    "hit_min_margin",
    F.col("recommended_price_raw") < F.col("min_price")
)

df_pricing = df_pricing.withColumn(
    "hit_max_price",
    F.col("recommended_price_raw") > F.col("max_price")
)

# Add constraint violation to reason codes
df_pricing = df_pricing.withColumn(
    "reason_codes",
    F.when(
        F.col("hit_min_margin"),
        F.array_union(F.col("reason_codes"), F.array(F.lit("CONSTRAINT_MIN_MARGIN")))
    ).otherwise(F.col("reason_codes"))
)

df_pricing = df_pricing.withColumn(
    "reason_codes",
    F.when(
        F.col("hit_max_price"),
        F.array_union(F.col("reason_codes"), F.array(F.lit("CONSTRAINT_MAX_PRICE")))
    ).otherwise(F.col("reason_codes"))
)

# Constraint 3: Max change per week (10%)
# For first run, we don't have previous prices, so this check is skipped
if df_previous_prices is not None:
    df_prev = df_previous_prices.select(
        "product_id",
        F.col("recommended_price").alias("previous_price"),
        F.col("recommendation_ts").alias("previous_ts")
    )
    
    df_pricing = df_pricing.join(df_prev, "product_id", "left")
    
    df_pricing = df_pricing.withColumn(
        "days_since_last_change",
        F.datediff(F.current_timestamp(), F.col("previous_ts"))
    )
    
    # Check if change exceeds 10% and duration < 7 days
    df_pricing = df_pricing.withColumn(
        "change_pct",
        F.abs((F.col("recommended_price") - F.col("previous_price")) / F.col("previous_price"))
    )
    
    df_pricing = df_pricing.withColumn(
        "violates_max_change",
        (F.col("change_pct") > MAX_CHANGE_PCT_WEEKLY) & 
        (F.col("days_since_last_change") < MIN_DURATION_DAYS)
    )
    
    # If violates, keep previous price
    df_pricing = df_pricing.withColumn(
        "recommended_price",
        F.when(
            F.col("violates_max_change") & F.col("previous_price").isNotNull(),
            F.col("previous_price")
        ).otherwise(F.col("recommended_price"))
    )
    
    df_pricing = df_pricing.withColumn(
        "reason_codes",
        F.when(
            F.col("violates_max_change"),
            F.array_union(F.col("reason_codes"), F.array(F.lit("CONSTRAINT_MAX_CHANGE")))
        ).otherwise(F.col("reason_codes"))
    )
else:
    df_pricing = df_pricing.withColumn("previous_price", F.lit(None).cast("decimal(10,2)"))
    df_pricing = df_pricing.withColumn("previous_ts", F.lit(None).cast("timestamp"))
    df_pricing = df_pricing.withColumn("days_since_last_change", F.lit(None).cast("int"))
    df_pricing = df_pricing.withColumn("change_pct", F.lit(None).cast("decimal(5,2)"))
    df_pricing = df_pricing.withColumn("violates_max_change", F.lit(False))

print(f"Constraints applied to {df_pricing.count()} products")
print()

In [ ]:
# =============================================================================
# CREATE OUTPUT TABLE
# =============================================================================

print("="*60)
print("CREATING PRICING RECOMMENDATIONS OUTPUT")
print("="*60)

# Add no-change reason for products that don't need markdown
df_pricing = df_pricing.withColumn(
    "reason_codes",
    F.when(
        F.size(F.col("reason_codes")) == 0,
        F.array(F.lit("NO_CHANGE"))
    ).otherwise(F.col("reason_codes"))
)

# Create final output with extensible schema for future ML phases
df_output = df_pricing.select(
    "product_id",
    "ProductName",
    "Department",
    "Category",
    "Subcategory",
    F.col("Cost").alias("cost"),
    F.col("MSRP").alias("msrp"),
    F.col("SalePrice").alias("current_price"),
    F.col("recommended_price"),
    F.round((F.col("recommended_price") - F.col("SalePrice")) / F.col("SalePrice") * 100, 2).alias("change_pct"),
    F.col("reason_codes"),
    F.col("markdown_factor"),
    F.col("total_inventory"),
    F.col("days_since_last_receipt"),
    F.col("hit_min_margin"),
    F.col("hit_max_price"),
    F.col("violates_max_change"),
    F.lit("RULE_BASED").alias("model_type"),
    F.lit(None).cast("decimal(5,4)").alias("ml_confidence"),  # Placeholder for Phase 2
    F.current_timestamp().alias("recommendation_ts"),
    F.lit("1.0").alias("schema_version")
)

save_gold(df_output, "gold_pricing_recommendations")
print()

In [ ]:
# =============================================================================
# SUMMARY STATISTICS
# =============================================================================

print("="*60)
print("PRICING RECOMMENDATIONS SUMMARY")
print("="*60)

total_products = df_output.count()
print(f"\nTotal products analyzed: {total_products}")

# Count by reason code
print("\nRecommendations by reason:")
df_reasons = df_output.select(F.explode("reason_codes").alias("reason"))
df_reasons.groupBy("reason").count().orderBy(F.desc("count")).show(truncate=False)

# Count constraint violations
print("\nConstraint violations:")
min_margin_hits = df_output.filter(F.col("hit_min_margin")).count()
max_price_hits = df_output.filter(F.col("hit_max_price")).count()
max_change_hits = df_output.filter(F.col("violates_max_change")).count()

print(f"  Min margin: {min_margin_hits} products")
print(f"  Max price: {max_price_hits} products")
print(f"  Max change/week: {max_change_hits} products")

# Price change distribution
print("\nPrice change distribution:")
df_output.select(
    F.avg("change_pct").alias("avg_change_pct"),
    F.min("change_pct").alias("min_change_pct"),
    F.max("change_pct").alias("max_change_pct")
).show()

# Products with significant markdowns
significant_markdowns = df_output.filter(F.col("change_pct") < -5).count()
print(f"\nProducts with >5% markdown: {significant_markdowns}")

print("\n" + "="*60)
print("DYNAMIC PRICING PHASE 1 COMPLETE")
print("="*60)
print("\nNext steps:")
print("  - Review recommendations in gold_pricing_recommendations table")
print("  - Validate constraint logic with business stakeholders")
print("  - Phase 2: Add ML model for demand forecasting")
print("  - Phase 3: Add competitor price intelligence")